In [1]:
import pandas as pd

In [2]:
input_data = pd.read_csv('/Users/ankur.kumar/Downloads/train.csv')

In [15]:
input_data = input_data[~input_data.text.isna()]

In [16]:
input_data.to_csv('/Users/ankur.kumar/Downloads/train.csv')

In [4]:
len(input_data)

27481

In [7]:
input_data.iloc[0]

textID                                     cb774db0d1
text              I`d have responded, if I were going
selected_text     I`d have responded, if I were going
sentiment                                     neutral
Name: 0, dtype: object

In [ ]:
examples = []

for i in range(len(input_data)):
    row = input_data.iloc[i]
    qas_id = row.textID
    question_text = row.sentiment
    paragraph_text = row.text.strip()
    orig_answer_text = row.selected_text.strip()
    start_position = paragraph_text.find(orig_answer_text)
    is_impossible = False 
    
    example = SquadExample(
            qas_id=qas_id,
            question_text=question_text,
            paragraph_text=paragraph_text,
            orig_answer_text=orig_answer_text,
            start_position=start_position,
            is_impossible=is_impossible)
    
    examples.append(example)
    

In [17]:
import tensorflow as tf

In [20]:
filename = '/Users/ankur.kumar/Desktop/Personal/Projects/git projects/SQuAD/data/train_tweet.tf_record'
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [22]:
for raw_record in raw_dataset.take(10):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

features {
  feature {
    key: "cls_index"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "end_positions"
    value {
      int64_list {
        value: 12
      }
    }
  }
  feature {
    key: "input_ids"
    value {
      int64_list {
        value: 2
        value: 8387
        value: 3
        value: 31
        value: 1
        value: 43
        value: 57
        value: 4567
        value: 15
        value: 100
        value: 31
        value: 46
        value: 228
        value: 3
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        v